# Areas of Interest to Django
Import SpatiaLite records to a GeoDjango database

### Import libraries, define settings, import model

In [ ]:
import os
import django
from django.db import transaction
from django.contrib.gis.geos import GEOSGeometry
import sqlite3
from asgiref.sync import sync_to_async

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from whale.models import AreaOfInterest

### User defined variables

In [ ]:
sdb = "C:/gis/gaia/data/databases/gaia.db"

### Perform async insertion to database

In [ ]:
@sync_to_async
def insert_aoi(id, name, wkt_geometry, sqkm):
    """ Insert Areas of Interest into the GeoDjango database"""
    geometry = GEOSGeometry(wkt_geometry)
    AreaOfInterest.objects.update_or_create(
        id = id,
        name = name,
        geometry = geometry,
        sqkm = sqkm
    )

async def load_spatialite_data():
    """ Connect to original SpatiaLite Database, select all records
            call insert function in sync_to_async mode.
    """
    conn = sqlite3.connect(sdb)
    conn.enable_load_extension(True)
    conn.execute("SELECT load_extension('mod_spatialite')")
    cursor = conn.cursor()

    cursor.execute("SELECT aid, name, sqkm, AsText(geom) from aoi")

    for row in cursor.fetchall():
        id, name, sqkm, wkt_geometry = row
        await insert_aoi(id, name, wkt_geometry, sqkm)

    conn.close()

await load_spatialite_data()

# End